In [2]:
import numpy as np
import pandas as pd

In [ ]:
# 3
def absolute_Distance(coords_1, coords_2):
    dist = np.sqrt((coords_2[1] - coords_1[1])**2 + (coords_2[2] - coords_1[2])**2 + (coords_2[3] - coords_1[3])**2)
    return dist

In [ ]:
# Inputs: x1 and x2 are lists that contains the x, y, and z coordinates of an atom
def lennard_Jones_PE(atom1, atom2):


In [ ]:
def tot_Energy():

